<a href="https://colab.research.google.com/github/waspar00/Gusev-2025/blob/main/LeNet-AlexNet-VGG(TODO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
#determining the device type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#defining training and evaluation procedures
def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99 and (epoch + 1) % 5 == 0:
                print(f'[E- {epoch + 1}, B- {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
    print('Done')

def evaluate_model(model, testloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Accuracy: {round(accuracy, 3) * 100}%')
    print(f'Score: {round(f1, 3)}')



In [5]:
#setting up and transforming a dataset/loaders
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
#defining lenet structure
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        c1 = F.relu(self.conv1(input))
        s2 = F.max_pool2d(c1, (2, 2))
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, 2)
        s4 = torch.flatten(s4, 1)
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))
        output = self.fc3(f6)
        return output

In [11]:
model = LeNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

[E- 5, B- 100] loss: 1.105
[E- 5, B- 200] loss: 1.105
[E- 5, B- 300] loss: 1.103
[E- 5, B- 400] loss: 1.094
[E- 5, B- 500] loss: 1.107
[E- 10, B- 100] loss: 0.843
[E- 10, B- 200] loss: 0.840
[E- 10, B- 300] loss: 0.830
[E- 10, B- 400] loss: 0.871
[E- 10, B- 500] loss: 0.864
Done
Accuracy: 63.0%
Score: 0.629


In [12]:
model = LeNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

[E- 5, B- 100] loss: 1.105
[E- 5, B- 200] loss: 1.132
[E- 5, B- 300] loss: 1.119
[E- 5, B- 400] loss: 1.135
[E- 5, B- 500] loss: 1.128
[E- 10, B- 100] loss: 0.999
[E- 10, B- 200] loss: 1.008
[E- 10, B- 300] loss: 1.014
[E- 10, B- 400] loss: 1.038
[E- 10, B- 500] loss: 1.049
Done
Accuracy: 58.699999999999996%
Score: 0.577


In [13]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)
        self.dropout = nn.Dropout()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
#testing alexnet
model = AlexNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

[E- 5, B- 100] loss: 0.728
[E- 5, B- 200] loss: 0.731
[E- 5, B- 300] loss: 0.726
[E- 5, B- 400] loss: 0.700
[E- 5, B- 500] loss: 0.705
[E- 10, B- 100] loss: 0.329
[E- 10, B- 200] loss: 0.376
[E- 10, B- 300] loss: 0.369
[E- 10, B- 400] loss: 0.368
[E- 10, B- 500] loss: 0.397
Done
Accuracy: 79.7%
Score: 0.796


TODO: Properly train the VGG thingamajig

In [ ]:
#testing vgg
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


[E- 5, B- 100] loss: 0.756
[E- 5, B- 200] loss: 0.758
[E- 5, B- 300] loss: 0.766
[E- 5, B- 400] loss: 0.720
[E- 5, B- 500] loss: 0.756
